# Задание

Сравните метрики hit_rate@k, precision@k.  
Какую метрику использовать предпочтительно и почему?  

### Ответ
**hit_rate@k** - менее информативна, т.к. отмечает лишь факт наличия успешной рекомендации. Может использоваться когда ожидается только одна покупка.

**precision@k** - более информативна, т.к. показывает степень успешности предложенных рекомендаций    
(в процентном соотношении сколько товаров из числа предложенных было куплено)

Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?  

1) Подбор цвета конкретной модели шапки по фотографии человека в онлайн магазине:  
**П** - фотография  
**Т** - шапки разных цветов  

2) Подбор цвета iphone 12 по профилю и истории покупок магазина:  
**П** - клиент  
**Т** - устройства разных цветов

## Задание

В метрике NDCG@k мы используем логарифм в знаменателе.  
Как Вы думаете, почему именно логарифм?  
Какую функцию можно использовать вместо логарифма?  
Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.  
**Precision, Recall, F-score, R2**

Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована).  

boughted = [1, 3, 5, 7, 9, 11], recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8